# Install Libraries

In [1]:
!pip install -q pymupdf sqlalchemy
!pip install -q mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 95.9 MB/s eta 0:00:00


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul 30 13:16:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   48C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Ordenanzas

In [109]:
import requests
# import pdfplumber
from io import BytesIO
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime
import fitz  # PyMuPDF

# Base URL and number of pages
base_url = "https://www.coruna.gal/obtenermasresultados/es?itemID=1453612770277&site=Transparencia"
num_pages = 6  # Number of pages to scrape

# Mapping category names to their corresponding categoriaID values
categories = {
    "Administración y gestión": "1453773633326",
    "Fiscales": "1453773634816",
    "Medio ambiente": "1453773637896",
    "Sanidad y consumo": "1453773640360",
    "Seguridad ciudadana": "1453773640660",
    "Servicios sociales": "1453773641299",
    "Transportes y circulación": "1453773642032",
    "Urbanismo": "1453773642338"
}

def extract_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Extracting all links
    links = [a['href'] for a in soup.find_all('a', href=True) if not a['href'].startswith('javascript:')]
    # Filter out links that end in .htm or .pdf
    filtered_links = [link for link in links if not link.lower().endswith(('.htm', '.html', '.pdf'))]
    # Prepend base URL to relative links
    full_links = [urljoin('https://www.coruna.gal/', link) for link in filtered_links]
    return full_links

def get_page_title(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else 'No Title'
        return title.strip()
    except Exception as e:
        print(f"Error fetching title for URL {url}: {e}")
        return 'Error'

def extract_pdf_links_and_titles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')
        pdf_links_with_titles = []

        # Find all <a> tags
        for a in soup.find_all('a', href=True):
            href = a['href']
            if href.lower().endswith('.pdf'):
                pdf_url = urljoin(url, href)
                # Extract the descriptive text
                pdf_title = a.get_text(strip=True)
                pdf_links_with_titles.append((pdf_url, pdf_title))
            elif href.lower().endswith('.htm'):
                htm_url = urljoin(url, href)
                htm_title = a.get_text(strip=True)
                pdf_links_with_titles.append((htm_url, htm_title))

        return pdf_links_with_titles
    except requests.RequestException as e:
        print(f"Request failed for {url}: {e}")
        return []


def extract_text_from_pdf(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with BytesIO(response.content) as pdf_file:
            with fitz.open(stream=pdf_file.read(), filetype="pdf") as pdf:
                text = ''.join(page.get_text() for page in pdf)
        return text
    except Exception as e:
        print(f"Error extracting text from PDF {url}: {e}")
        return 'Error'

def extract_text_from_htm(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator='\n', strip=True)
        return text
    except requests.RequestException as e:
        print(f"Error extracting text from HTM {url}: {e}")
        return 'Error'
    except Exception as e:
        print(f"Error extracting text from HTM {url}: {e}")
        return 'Error'

# Initialize lists to hold category, URLs, titles, PDF URLs, PDF titles and content
category_list = []  # grupo
all_urls = []       # Ordenanza url
titles = []         # Ordenanza name (subgrupo)

# Get current date
current_date = datetime.today().strftime('%Y-%m-%d')

# Iterate over all categories and pages
for category, categoria_id in categories.items():
    for page_num in range(1, num_pages + 1):
        page_url = f"{base_url}&categoriaID={categoria_id}&argPag={page_num}#Componente1453612770277"
        print(f"Scraping category '{category}' page {page_num}...")
        urls = extract_info(page_url)
        category_list.extend([category] * len(urls))
        all_urls.extend(urls)
        # Get titles for the URLs
        for url in urls:
            titles.append(get_page_title(url))

# Create a DataFrame
df = pd.DataFrame({
    'ciudad': 'A Coruña',
    'date': current_date,
    'grupo': category_list,
    'url_subgrupo': all_urls,
    'subgrupo': titles,

})

# Initialize lists to hold the expanded rows
expanded_data = []

# Iterate over the DataFrame and extract PDF links and titles
for index, row in df.iterrows():
    url = row['url_subgrupo']
    pdfs_with_titles = extract_pdf_links_and_titles(url)
    if not pdfs_with_titles:
        # If no PDFs are found, just append the row as is
        expanded_data.append({
            'ciudad': 'A Coruña',
            'date': current_date,
            'titulo': None,
            'grupo': row['grupo'],
            'subgrupo': row['subgrupo'],
            'url': None,
            'content': None,
            'url_subgrupo': row['url_subgrupo'],

        })
    else:
        # If PDFs are found, create a new row for each PDF
        for file_url, file_title in pdfs_with_titles:
            if file_url.lower().endswith('.pdf'):
                file_text = extract_text_from_pdf(file_url)
            else:
                file_text = extract_text_from_htm(file_url)

            expanded_data.append({
                'ciudad': 'A Coruña',
                'date': current_date,
                'titulo': file_title,
                'grupo': row['grupo'],
                'subgrupo': row['subgrupo'],
                'url': file_url,
                'content': file_text,
                'url_subgrupo': row['url_subgrupo'],
            })

Scraping category 'Administración y gestión' page 1...
Scraping category 'Administración y gestión' page 2...
Scraping category 'Administración y gestión' page 3...
Scraping category 'Administración y gestión' page 4...
Scraping category 'Administración y gestión' page 5...
Scraping category 'Administración y gestión' page 6...
Scraping category 'Fiscales' page 1...
Scraping category 'Fiscales' page 2...
Scraping category 'Fiscales' page 3...
Scraping category 'Fiscales' page 4...
Scraping category 'Fiscales' page 5...
Scraping category 'Fiscales' page 6...
Scraping category 'Medio ambiente' page 1...
Scraping category 'Medio ambiente' page 2...
Scraping category 'Medio ambiente' page 3...
Scraping category 'Medio ambiente' page 4...
Scraping category 'Medio ambiente' page 5...
Scraping category 'Medio ambiente' page 6...
Scraping category 'Sanidad y consumo' page 1...
Scraping category 'Sanidad y consumo' page 2...
Scraping category 'Sanidad y consumo' page 3...
Scraping category 'San

In [110]:
# Create a DataFrame for the expanded data
df_ordenanzas = pd.DataFrame(expanded_data)

In [119]:
df_ordenanzas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ciudad        166 non-null    object
 1   date          166 non-null    object
 2   titulo        166 non-null    object
 3   grupo         166 non-null    object
 4   subgrupo      166 non-null    object
 5   url           166 non-null    object
 6   content       166 non-null    object
 7   url_subgrupo  166 non-null    object
dtypes: object(8)
memory usage: 10.5+ KB


In [120]:
import pandas as pd
import logging

# Save the DataFrame to a Parquet file
try:
    df_ordenanzas.to_parquet(
        '/content/drive/MyDrive/Colab Notebooks/14 Scraping/A_Coruna/a_coruna_ordenanzas.parquet',
        index=False
    )
    print("DataFrame created and saved to 'a_coruna_ordenanzas.parquet'")
except Exception as e:
    logging.error(f"Error saving DataFrame to Parquet: {e}")


DataFrame created and saved to 'a_coruna_ordenanzas.parquet'


In [121]:
# Load the DataFrame from the Parquet file
try:
    df_loaded = pd.read_parquet(
        '/content/drive/MyDrive/Colab Notebooks/14 Scraping/A_Coruna/a_coruna_ordenanzas.parquet'
    )
    print("DataFrame loaded successfully")
except Exception as e:
    logging.error(f"Error loading DataFrame from Parquet: {e}")

# Check the DataFrame info
df_loaded.info()


DataFrame loaded successfully
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ciudad        166 non-null    object
 1   date          166 non-null    object
 2   titulo        166 non-null    object
 3   grupo         166 non-null    object
 4   subgrupo      166 non-null    object
 5   url           166 non-null    object
 6   content       166 non-null    object
 7   url_subgrupo  166 non-null    object
dtypes: object(8)
memory usage: 10.5+ KB


# Reglamentos

In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import fitz  # PyMuPDF
from io import BytesIO

def extract_text_from_pdf(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with BytesIO(response.content) as pdf_file:
            with fitz.open(stream=pdf_file.read(), filetype="pdf") as pdf:
                text = ''.join(page.get_text() for page in pdf)
        return text
    except Exception as e:
        print(f"Error extracting text from PDF {url}: {e}")
        return 'Error'

def extract_text_from_htm(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator='\n', strip=True)
        return text
    except requests.RequestException as e:
        print(f"Error extracting text from HTM {url}: {e}")
        return 'Error'
    except Exception as e:
        print(f"Error extracting text from HTM {url}: {e}")
        return 'Error'

def get_file_links(url, base_url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    file_links = []

    # Find all <a> tags with href attributes
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        text = a_tag.get_text(strip=True)

        # Check if href ends with .htm or .pdf
        if href.endswith('.htm') or href.endswith('.pdf'):
            full_url = base_url + href if not href.startswith('http') else href
            file_links.append([text, url , full_url])

    return file_links

def get_file_content(row):
    file_url = row['url']
    if file_url.endswith('.pdf'):
        return extract_text_from_pdf(file_url)
    elif file_url.endswith('.htm'):
        return extract_text_from_htm(file_url)
    return 'Not a valid file type'

def determine_grupo(url):
    if "organico" in url.lower():
        return "Reglamento Organico"
    else:
        return "Reglamento General"

# URL of the main webpage
url = "https://www.coruna.gal/transparencia/es/normativa-municipal/normativa?_charset_=UTF-8&categoriaTipo=Contenido&categoriaID=1453696054145&textoBuscar=&buscar=Buscar"

# Base URL to be added to each href
base_url = "https://www.coruna.gal"

# Define headers to mimic a web browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Make a request to the main webpage with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <h1> elements with property='name'
h1_elements = soup.find_all('h1', property='name')

# Extract href and text
data = []
for h1 in h1_elements:
    a_tag = h1.find('a')
    if a_tag:
        href = a_tag.get('href')
        text = a_tag.get_text(strip=True)
        full_url = base_url + href

        # Get file links from the URL
        file_links = get_file_links(full_url, base_url)
        data.extend(file_links)

# Create a DataFrame
df_reg = pd.DataFrame(data, columns=['titulo', 'url_subgrupo', 'url'])

# Add content extraction
df_reg['content'] = df_reg.apply(get_file_content, axis=1)

# Add the 'grupo' column based on the presence of 'organico' in the Page URL
df_reg['grupo'] = df_reg['url_subgrupo'].apply(determine_grupo)

ciudad = "A Coruña"
current_date = datetime.today().strftime('%Y-%m-%d')

df_reg.insert(0, 'ciudad', ciudad)  # Insert 'ciudad' as the first column

df_reg.insert(1, 'date', current_date)  # Insert 'ciudad' as the first column

df_reg.insert(3, 'subgrupo', "Reglamentos")  # Insert 'ciudad' as the first column

# Reorder columns to match the desired order
df_reg = df_reg[['ciudad', 'date', 'titulo', "grupo", 'subgrupo', 'url', 'content', 'url_subgrupo']]

# Display the DataFrame
df_reg

,ciudad,date,titulo,grupo,subgrupo,url,content,url_subgrupo
0,A Coruña,2024-07-30,Regulamento Regulador do Fondo de Acción Socia...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90010228508640...,,https://www.coruna.gal/transparencia/gl/normat...
1,A Coruña,2024-07-30,Plan Disposición Fondos (108 KB),Reglamento General,Reglamentos,https://www.coruna.gal/descarga/1453788565479/...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...,https://www.coruna.gal/transparencia/gl/normat...
2,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/20...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...,https://www.coruna.gal/transparencia/gl/normat...
3,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/19...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/19...,,https://www.coruna.gal/transparencia/gl/normat...
4,A Coruña,2024-07-30,Regulamento Honras Distincións (2009) (Castelá...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90011468325687...,1\nREGLAMENTO PARA LA CONCESIÓN DE HONORES Y D...,https://www.coruna.gal/transparencia/gl/normat...
5,A Coruña,2024-07-30,Regulamento Honras Distincións (2009) (Galego)...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90011468325687...,\n1 \nREGULAMENTO PARA A CONCESIÓN DE HONORES...,https://www.coruna.gal/transparencia/gl/normat...
6,A Coruña,2024-07-30,estacion_autobuses.pdf (118 KB),Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90010228508593...,"5(*/$0(172'((;3/27$&,21'(/$(67$&,21'(\n...",https://www.coruna.gal/transparencia/gl/normat...
7,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/20...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...,https://www.coruna.gal/transparencia/gl/normat...
8,A Coruña,2024-07-30,Regulamento%20de%20rexime%20interior%20do%20Ce...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90010228508671...,1\nAÑO 1.980\n Negociado de Cem...,https://www.coruna.gal/transparencia/gl/normat...
9,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/19...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/19...,"ANUNCIO - Boletín Nº 25. Lunes, 1 de febrero d...",https://www.coruna.gal/transparencia/gl/normat...


In [105]:
# Dropping rows 0, 3, 5 and 20. 0 is not a reglamento and 3 is very old has a consolidated/updated pdf in row 4. Indexes 5 and 20 are galician versions

df_reg = df_reg.drop([0, 3, 5, 20])

df_reg.reset_index(drop=True, inplace=True)

df_reg

,ciudad,date,titulo,grupo,subgrupo,url,content,url_subgrupo
0,A Coruña,2024-07-30,Plan Disposición Fondos (108 KB),Reglamento General,Reglamentos,https://www.coruna.gal/descarga/1453788565479/...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...,https://www.coruna.gal/transparencia/gl/normat...
1,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/20...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...,https://www.coruna.gal/transparencia/gl/normat...
2,A Coruña,2024-07-30,Regulamento Honras Distincións (2009) (Castelá...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90011468325687...,1\nREGLAMENTO PARA LA CONCESIÓN DE HONORES Y D...,https://www.coruna.gal/transparencia/gl/normat...
3,A Coruña,2024-07-30,estacion_autobuses.pdf (118 KB),Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90010228508593...,"5(*/$0(172'((;3/27$&,21'(/$(67$&,21'(\n...",https://www.coruna.gal/transparencia/gl/normat...
4,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/20...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...,https://www.coruna.gal/transparencia/gl/normat...
5,A Coruña,2024-07-30,Regulamento%20de%20rexime%20interior%20do%20Ce...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/90010228508671...,1\nAÑO 1.980\n Negociado de Cem...,https://www.coruna.gal/transparencia/gl/normat...
6,A Coruña,2024-07-30,https://bop.dicoruna.es/bopportal/publicado/19...,Reglamento General,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/19...,"ANUNCIO - Boletín Nº 25. Lunes, 1 de febrero d...",https://www.coruna.gal/transparencia/gl/normat...
7,A Coruña,2024-07-30,Regulamento de organización e funcionamento da...,Reglamento General,Reglamentos,https://www.coruna.gal/descarga/1453821265765/...,\nConcello da Coruña - Plaza de María Pita 1 ...,https://www.coruna.gal/transparencia/gl/normat...
8,A Coruña,2024-07-30,BOP de 13 de decembro de 2004,Reglamento Organico,Reglamentos,https://bop.dicoruna.es/bopportal/publicado/20...,"EDICTO - Boletín Nº 285. Lunes, 13 de diciembr...",https://www.coruna.gal/transparencia/gl/normat...
9,A Coruña,2024-07-30,Regulamento orgánico de participación cidadá (...,Reglamento Organico,Reglamentos,https://www.coruna.gal/descarga/1171021491373/...,\n1\n \nREGLAMENTO ORGÁNICO DE \nPARTICIPACIÓ...,https://www.coruna.gal/transparencia/gl/normat...


In [106]:
df_reg.to_csv('/content/drive/MyDrive/Colab Notebooks/14 Scraping/A_Coruna/a_coruna_reglamentos.csv', index=False)

# Concatenate PDFs

In [122]:
df_ordenanzas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ciudad        166 non-null    object
 1   date          166 non-null    object
 2   titulo        166 non-null    object
 3   grupo         166 non-null    object
 4   subgrupo      166 non-null    object
 5   url           166 non-null    object
 6   content       166 non-null    object
 7   url_subgrupo  166 non-null    object
dtypes: object(8)
memory usage: 10.5+ KB


In [123]:
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ciudad        17 non-null     object
 1   date          17 non-null     object
 2   titulo        17 non-null     object
 3   grupo         17 non-null     object
 4   subgrupo      17 non-null     object
 5   url           17 non-null     object
 6   content       17 non-null     object
 7   url_subgrupo  17 non-null     object
dtypes: object(8)
memory usage: 1.2+ KB


In [124]:
final_df_coruna = pd.concat([df_ordenanzas, df_reg], axis=0, ignore_index=True)
final_df_coruna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ciudad        183 non-null    object
 1   date          183 non-null    object
 2   titulo        183 non-null    object
 3   grupo         183 non-null    object
 4   subgrupo      183 non-null    object
 5   url           183 non-null    object
 6   content       183 non-null    object
 7   url_subgrupo  183 non-null    object
dtypes: object(8)
memory usage: 11.6+ KB


# Create SQL

In [125]:
!apt-get update
!apt-get install -y mysql-server mysql-client
!service mysql start
!mysql -e "CREATE DATABASE testdb;"
!mysql -e "CREATE USER 'testuser'@'localhost' IDENTIFIED BY 'testpassword';"
!mysql -e "GRANT ALL PRIVILEGES ON testdb.* TO 'testuser'@'localhost';"
!mysql -e "GRANT PROCESS, RELOAD, SHOW DATABASES ON *.* TO 'testuser'@'localhost';"
!mysql -e "FLUSH PRIVILEGES;"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [

In [126]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import BytesIO
import mysql.connector

# Configuración de la conexión a MySQL
mydb = mysql.connector.connect(
    host="localhost",
    user="testuser",
    password="testpassword",
    database="testdb"
)

mycursor = mydb.cursor()


# Create the table in the database if it doesn't exist and adjust column types
create_table_query = """
CREATE TABLE IF NOT EXISTS normativa (
    id INT AUTO_INCREMENT PRIMARY KEY,
    ciudad VARCHAR(255),
    date DATE,
    titulo TEXT,
    grupo VARCHAR(255),
    subgrupo TEXT,
    url TEXT,
    content MEDIUMTEXT
)
"""

mycursor.execute(create_table_query)

# Insertar los datos en la tabla de MySQL
try:
  for index, row in final_df_coruna.iterrows():
      sql = "INSERT INTO normativa (ciudad, date, titulo, grupo, subgrupo, url, content) VALUES (%s, %s, %s, %s, %s, %s, %s)"
      val = (row['ciudad'], row['date'], row['titulo'], row['grupo'], row['subgrupo'], row['url'], row['content'])

      mycursor.execute(sql, val)
      print(row['titulo'], " insertado en tabla.")

  mydb.commit()
  print(mycursor.rowcount, "registro(s) insertado(s).")

except mysql.connector.Error as err:
  print("Error: {}".format(err))
# finally:
#     # Close the cursor and connection
#     mycursor.close()
#     mydb.close()

BOP núm 114 de 17 de xuño de 2023  insertado en tabla.
Certificado acordo plenario aprobación definitiva   (6 MB)  insertado en tabla.
Aprobación definitiva da ordenanza reguladora da administración dixital no Concello da Coruña – BOP   (312 KB)  insertado en tabla.
Emenda de erro na publicación do texto articulado da ordenanza reguladora da administración dixital no Concello da Coruña – BOP   (123 KB)  insertado en tabla.
BOP núm. 116 do martes 22 de xuño de 2021  insertado en tabla.
Ordenanza do uso da lingua no Concello da Coruña (119 KB)  insertado en tabla.
LIBRO-ORDENANZAS-2024.pdf (2 MB)  insertado en tabla.
Ordenanza fiscal núm. 01-Reguladora da taxa por expedición de documentos (58 KB)  insertado en tabla.
Ordenanza Fiscal nº 02-Reguladora da taxa pola prestación de servizos de vixilancia de transportes. (46 KB)  insertado en tabla.
Ordenanza Fiscal nº 03-Reguladora da taxa polo otorgamiento de licenzas e outros servizos urbanísticos. (68 KB)  insertado en tabla.
Ordenanza Fis

In [127]:
# Query the database
query = "SELECT * FROM normativa"
df = pd.read_sql(query, mydb)

# Display the DataFrame
df.head()

<ipython-input-127-b7a4fd91b2d3>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


,id,ciudad,date,titulo,grupo,subgrupo,url,content
0,1,A Coruña,2024-07-30,BOP núm 114 de 17 de xuño de 2023,Administración y gestión,Ordenanza reguladora da administración dixital...,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...
1,2,A Coruña,2024-07-30,Certificado acordo plenario aprobación definit...,Administración y gestión,Ordenanza reguladora da administración dixital...,https://www.coruna.gal/descarga/1453839849193/...,Concello da Coruña - Plaza de María Pita 1 - 9...
2,3,A Coruña,2024-07-30,Aprobación definitiva da ordenanza reguladora ...,Administración y gestión,Ordenanza reguladora da administración dixital...,https://www.coruna.gal/descarga/1453839849294/...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...
3,4,A Coruña,2024-07-30,Emenda de erro na publicación do texto articul...,Administración y gestión,Ordenanza reguladora da administración dixital...,https://www.coruna.gal/descarga/1453839849665/...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...
4,5,A Coruña,2024-07-30,BOP núm. 116 do martes 22 de xuño de 2021,Administración y gestión,Ordenanza reguladora do uso da lingua galega n...,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        183 non-null    int64 
 1   ciudad    183 non-null    object
 2   date      183 non-null    object
 3   titulo    183 non-null    object
 4   grupo     183 non-null    object
 5   subgrupo  183 non-null    object
 6   url       183 non-null    object
 7   content   183 non-null    object
dtypes: int64(1), object(7)
memory usage: 11.6+ KB


In [129]:
# Export the database to a SQL dump file
!mysqldump -u testuser -ptestpassword testdb > "/content/drive/My Drive/Colab Notebooks/14 Scraping/A_Coruna/a_coruna.sql"

mysqldump: [Warning] Using a password on the command line interface can be insecure.


In [130]:
import pandas as pd
from sqlalchemy import create_engine

# Define database connection parameters
db_config = {
    "host":"localhost",
    "user":"testuser",
    "password":"testpassword",
    "database":"testdb"
}

# Establish the connection
connection_url = f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"

# Create an SQLAlchemy engine
engine = create_engine(connection_url)

# Define the SQL query to select data
query = "SELECT * FROM normativa"

# Load data into a DataFrame
df = pd.read_sql(query, engine)

# Print the DataFrame
df.head()


,id,ciudad,date,titulo,grupo,subgrupo,url,content
0,1,A Coruña,2024-07-30,BOP núm 114 de 17 de xuño de 2023,Administración y gestión,Ordenanza reguladora da administración dixital...,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...
1,2,A Coruña,2024-07-30,Certificado acordo plenario aprobación definit...,Administración y gestión,Ordenanza reguladora da administración dixital...,https://www.coruna.gal/descarga/1453839849193/...,Concello da Coruña - Plaza de María Pita 1 - 9...
2,3,A Coruña,2024-07-30,Aprobación definitiva da ordenanza reguladora ...,Administración y gestión,Ordenanza reguladora da administración dixital...,https://www.coruna.gal/descarga/1453839849294/...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...
3,4,A Coruña,2024-07-30,Emenda de erro na publicación do texto articul...,Administración y gestión,Ordenanza reguladora da administración dixital...,https://www.coruna.gal/descarga/1453839849665/...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...
4,5,A Coruña,2024-07-30,BOP núm. 116 do martes 22 de xuño de 2021,Administración y gestión,Ordenanza reguladora do uso da lingua galega n...,https://bop.dicoruna.es/bopportal/publicado/20...,BOP\nBOLETÍN OFICIAL DA PROVINCIA DA CORUÑA BO...


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        183 non-null    int64 
 1   ciudad    183 non-null    object
 2   date      183 non-null    object
 3   titulo    183 non-null    object
 4   grupo     183 non-null    object
 5   subgrupo  183 non-null    object
 6   url       183 non-null    object
 7   content   183 non-null    object
dtypes: int64(1), object(7)
memory usage: 11.6+ KB
